<a href="https://colab.research.google.com/github/joseph7104/-1INF46-Plan_Compras_Produccion/blob/master/notebooks/Data_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/joseph7104/-1INF46-Plan_Compras_Produccion.git
%cd "./-1INF46-Plan_Compras_Produccion"

Cloning into '-1INF46-Plan_Compras_Produccion'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 61 (delta 10), reused 46 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (61/61), 4.62 MiB | 15.42 MiB/s, done.
Resolving deltas: 100% (10/10), done.
/content/-1INF46-Plan_Compras_Produccion


# **Generador de datos**

In [2]:
# ===============================================
#  NOTEBOOK: Simulación integral PYME gastronómica
#  Ventas → Producción → Capacidad → Compras → Inventario → Planificación
#  (c) Joseph & Proyecto de tesis
# ===============================================

import numpy as np, pandas as pd, random, string, math
from datetime import datetime, timedelta
from collections import defaultdict
import os

# -------------------------
# 1) VARIABLES GLOBALES
# -------------------------
SEED                 = 42
START_DATE           = "2021-01-01"
END_DATE             = "2025-12-31"     # puedes cambiarlo
FREQ                 = "D"              # diaria
IGV                  = 1.0              # multiplicador (1.0 = sin IGV; pon 1.18 si deseas incluir)
FOOD_COST_PCT        = 0.33             # Costo materia prima (% del precio)
MERMA_PROD_MIN       = 0.04             # merma en producción min
MERMA_PROD_MAX       = 0.10             # merma en producción max
MERMA_ALM_MENSUAL    = 0.015            # merma por almacenamiento mensual aproximada
MARGEN_BRUTO_META    = 0.55
COBERTURA_OBJ_DIAS   = 7                # objetivo de días de cobertura para compras
STOCK_MIN_DIAS       = 3                # stock mínimo ≈ 3 días de consumo
PANDEMIA_2021_MULT   = (0.7, 0.9)       # reduce demanda (rango aleatorio) para 2021
VENTAS_BASE_MIN_MAX  = (90, 180)        # platos/día en PYME (ajustado por calendario/eficiencia)
POP_PLATOS           = { # mix de popularidad (suma 1.0)
    "Lomo Saltado": 0.18, "Pollo a la Brasa": 0.16, "Ceviche Clásico": 0.12,
    "Arroz Chaufa": 0.14, "Tallarines Verdes": 0.10, "Ají de Gallina": 0.08,
    "Anticuchos": 0.06, "Seco de Res": 0.05, "Causa Limeña": 0.04,
    "Papa a la Huancaína": 0.03, "Chicharrón de Pescado": 0.02, "Sopa Criolla": 0.02
}
PRECIOS_POR_PLATO    = { # S/ unitario (se multiplica por IGV)
    "Lomo Saltado": 26, "Pollo a la Brasa": 32, "Ceviche Clásico": 30,
    "Arroz Chaufa": 24, "Tallarines Verdes": 25, "Ají de Gallina": 24,
    "Anticuchos": 22, "Seco de Res": 28, "Causa Limeña": 18,
    "Papa a la Huancaína": 15, "Chicharrón de Pescado": 28, "Sopa Criolla": 18
}
# Work center base (capacidad min/día, unidades paralelas y eficiencia con ruido)
WC_BASE = [
    ("WC_SARTEN",   "Sartén",   "Estacion", "Caliente", 480, 2, 0.90),
    ("WC_PLANCHA",  "Plancha",  "Estacion", "Caliente", 480, 1, 0.90),
    ("WC_HORNO",    "Horno",    "Equipo",   "Caliente", 480, 1, 0.90),
    ("WC_TABLA",    "Tabla",    "Estacion", "Fria",     480, 2, 0.92),
    ("WC_EMPLATE",  "Emplatado","Estacion", "General",  480, 2, 0.95),
]

# Rutas de salida
OUT_DIR = "data/raw"
os.makedirs(OUT_DIR, exist_ok=True)

random.seed(SEED); np.random.seed(SEED)

# -------------------------
# 2) CATÁLOGOS BÁSICOS
# -------------------------
lineas = pd.DataFrame([
    [1, "Caliente"],
    [2, "Fría"],
    [3, "General"]
], columns=["id_linea","nombre_linea"])

platos = pd.DataFrame([
    [1,"Lomo Saltado",1],
    [2,"Pollo a la Brasa",1],
    [3,"Ceviche Clásico",2],
    [4,"Arroz Chaufa",1],
    [5,"Tallarines Verdes",1],
    [6,"Ají de Gallina",1],
    [7,"Anticuchos",1],
    [8,"Seco de Res",1],
    [9,"Causa Limeña",2],
    [10,"Papa a la Huancaína",2],
    [11,"Chicharrón de Pescado",1],
    [12,"Sopa Criolla",1]
], columns=["id_plato","plato","id_linea"])

# Insumos realistas + unidad base
insumos_data = [
    [101,"Carne de res","kg"],
    [102,"Papa amarilla","kg"],
    [103,"Cebolla roja","kg"],
    [104,"Pollo entero","kg"],
    [105,"Papa blanca","kg"],
    [106,"Ensalada (mix)","kg"],
    [107,"Pescado blanco","kg"],
    [108,"Limón","kg"],
    [109,"Ají amarillo","kg"],
    [110,"Arroz","kg"],
    [111,"Huevo","unid"],
    [112,"Pollo desmenuzado","kg"],
    [113,"Fideos","kg"],
    [114,"Espinaca","kg"],
    [115,"Queso fresco","kg"],
    [116,"Ajo","kg"],
    [117,"Aceite vegetal","L"],
    [118,"Vinagre","L"],
    [119,"Leche","L"],
    [120,"Pan (para causa)","kg"],
    [121,"Harina","kg"],
    [122,"Chuño","kg"]
]
insumo = pd.DataFrame(insumos_data, columns=["id_insumo","nombre_insumo","unidad"])

# Receta (cantidades por porción de plato; unidades coherentes con insumo)
receta_rows = [
    # id_plato, id_insumo, cantidad, unidad_de_medida
    # Lomo Saltado
    [1,101,0.18,"kg"], [1,102,0.15,"kg"], [1,103,0.04,"kg"], [1,116,0.01,"kg"], [1,117,0.02,"L"],
    # Pollo a la Brasa
    [2,104,0.40,"kg"], [2,105,0.20,"kg"], [2,106,0.10,"kg"], [2,117,0.02,"L"],
    # Ceviche
    [3,107,0.20,"kg"], [3,108,0.05,"kg"], [3,103,0.04,"kg"],
    # Chaufa
    [4,110,0.25,"kg"], [4,111,1,"unid"], [4,112,0.06,"kg"], [4,117,0.015,"L"],
    # Tallarines Verdes
    [5,113,0.20,"kg"], [5,114,0.06,"kg"], [5,115,0.03,"kg"], [5,117,0.01,"L"],
    # Ají de Gallina
    [6,112,0.18,"kg"], [6,109,0.03,"kg"], [6,119,0.05,"L"], [6,110,0.18,"kg"],
    # Anticuchos
    [7,101,0.16,"kg"], [7,117,0.015,"L"], [7,109,0.02,"kg"],
    # Seco de Res
    [8,101,0.20,"kg"], [8,110,0.20,"kg"], [8,116,0.01,"kg"],
    # Causa Limeña
    [9,102,0.22,"kg"], [9,115,0.02,"kg"], [9,120,0.07,"kg"], [9,117,0.01,"L"],
    # Papa a la Huancaína
    [10,105,0.22,"kg"], [10,115,0.03,"kg"], [10,109,0.02,"kg"],
    # Chicharrón de Pescado
    [11,107,0.18,"kg"], [11,121,0.02,"kg"], [11,117,0.02,"L"],
    # Sopa Criolla
    [12,110,0.12,"kg"], [12,112,0.04,"kg"], [12,122,0.02,"kg"]
]
receta = pd.DataFrame(receta_rows, columns=["id_plato","id_insumo","cantidad","unidad_de_medida"])
receta["id_receta"] = range(1, len(receta)+1)
receta = receta[["id_receta","id_plato","id_insumo","cantidad","unidad_de_medida"]].assign(observaciones="")

# -------------------------
# 3) ALMACÉN (PYME: 1 almacén + 2 zonas: Seco/Frío)
# -------------------------
almacen = pd.DataFrame([[1,"Almacén Principal","Av. Siempre Viva 123", 1500.0]],
                       columns=["id_almacen","nombre","ubicacion","capacidad_total_kg"])
almacen_ubicacion = pd.DataFrame([
    [1,1,"SECO-A1",400.0, 0.0, "Activa"],
    [2,1,"SECO-A2",400.0, 0.0, "Activa"],
    [3,1,"FRIO-F1",300.0, 0.0, "Activa"],
    [4,1,"FRIO-F2",300.0, 0.0, "Activa"]
], columns=["id_ubicacion","id_almacen","codigo","capacidad_kg","espacio_usado_kg","estado"])

# Mapeo simple insumo→tipo ubicación (frío vs seco)
INSUMO_FRIO = set([104,107,111,115,119])  # pollo, pescado, huevo, queso, leche
def asignar_ubicacion_insumo(iid):
    # preferencia por frío si perecible
    if iid in INSUMO_FRIO:
        return random.choice([3,4])  # frío
    return random.choice([1,2])      # seco

# -------------------------
# 4) WORK CENTERS y HOJA DE RUTA (tiempos realistas)
# -------------------------
work_center = pd.DataFrame(WC_BASE,
                           columns=["work_center_id","work_center_name","tipo","linea","min_disp_dia","unidades_paralelas","eficiencia"])

ruta = []
def add_step(plato_id, plato_name, seq, wc_id, etapa, run_min, setup_min=1.0):
    wc_name = work_center.loc[work_center.work_center_id==wc_id,"work_center_name"].iloc[0]
    ruta.append([plato_id,plato_name,seq,wc_id,wc_name,etapa,run_min,setup_min])

name_by_id = dict(zip(platos.id_plato, platos.plato))
# Definir etapas simples por plato
for pid in platos.id_plato:
    nm = name_by_id[pid]
    # preparación fría si aplica
    if pid in [3,9,10]:  # ceviche, causa, huancaína
        add_step(pid,nm,10,"WC_TABLA","Mise en place", 1.5, 0.5)
        add_step(pid,nm,20,"WC_EMPLATE","Emplatado", 0.8, 0.2)
    # horno para pollo a la brasa
    elif pid==2:
        add_step(pid,nm,10,"WC_HORNO","Horneado", 6.0, 5.0)
        add_step(pid,nm,20,"WC_EMPLATE","Emplatado", 0.8,0.2)
    # caliente salteado/plancha/sartén
    elif pid in [1,4,5,6,7,8,11,12]:
        add_step(pid,nm,10,"WC_SARTEN","Salteado/Cocción", 3.5, 2.0)
        add_step(pid,nm,20,"WC_EMPLATE","Emplatado", 0.8, 0.2)

hoja_ruta = pd.DataFrame(ruta, columns=[
    "item_id","item_name","op_seq","work_center_id","work_center_name","etapa","run_time_min_por_unidad","setup_time_min"
])

# RCCP estándar (horas por unidad; suma pasos/60)
rccp_standard = (hoja_ruta.groupby(["item_id","item_name","work_center_id","work_center_name"], as_index=False)
                 .agg(run_time_min_por_unidad=("run_time_min_por_unidad","sum")))
rccp_standard["std_hours_per_unit"] = rccp_standard["run_time_min_por_unidad"]/60.0

# -------------------------
# 5) CALENDARIO PERÚ (2021–2025) con factores aleatorios
# -------------------------
fechas = pd.date_range(START_DATE, END_DATE, freq=FREQ)
cal_rows = []
FERIADOS_FIJOS = {
    # fecha (MM-DD): evento
    "01-01":"Año Nuevo","03-29":"Viernes Santo","03-30":"Sábado Santo","05-01":"Día del Trabajador",
    "06-29":"San Pedro y San Pablo","07-28":"Fiestas Patrias 1","07-29":"Fiestas Patrias 2",
    "08-30":"Santa Rosa de Lima","10-08":"Combate de Angamos","11-01":"Día de Todos los Santos",
    "12-08":"Inmaculada Concepción","12-25":"Navidad"
}
EVENTOS_VARIABLES = ["Día de la Madre","Día del Padre","Día del Pollo a la Brasa","Día del Ceviche"]

for f in fechas:
    feriado = 0; evento = ""; clima = random.choice(["Soleado","Nublado","Lluvioso"])
    key = f.strftime("%m-%d")
    if key in FERIADOS_FIJOS:
        feriado = 1; evento = FERIADOS_FIJOS[key]
    # eventos variables (de manera aleatoria leve)
    if f.month in [5,6] and random.random()<0.03:
        evento = random.choice(EVENTOS_VARIABLES)
    cal_rows.append([f, f.strftime("%A"), f.month, feriado, evento, clima, ""])
calendario_peru = pd.DataFrame(cal_rows, columns=[
    "fecha","dia_semana","mes","feriado","evento","clima","descripcion"
])

# Factores de demanda por calendario (rango aleatorio por día)
def factor_calendario(row):
    mult = 1.0
    dow = pd.Timestamp(row["fecha"]).dayofweek
    if dow>=5:  # fin de semana
        mult *= np.random.uniform(1.12,1.25)
    if row["feriado"]==1:
        mult *= np.random.uniform(1.25,1.45)
    if "Madre" in row["evento"] or "Padre" in row["evento"]:
        mult *= np.random.uniform(1.35,1.65)
    if "Pollo" in row["evento"] or "Ceviche" in row["evento"]:
        mult *= np.random.uniform(1.20,1.40)
    if row["clima"]=="Lluvioso":
        mult *= np.random.uniform(0.88,0.97)
    return mult

calendario_peru["factor_demanda"] = calendario_peru.apply(factor_calendario, axis=1)

# Pandemia 2021: reducir aleatoriamente
calendario_peru["factor_pandemia"] = 1.0
mask_2021 = (calendario_peru["fecha"].dt.year==2021)
calendario_peru.loc[mask_2021,"factor_pandemia"] = np.random.uniform(PANDEMIA_2021_MULT[0], PANDEMIA_2021_MULT[1], mask_2021.sum())

# -------------------------
# 6) Simulación VENTAS
# -------------------------
# preparar arrays auxiliares
plato_ids = platos.id_plato.tolist()
plato_names = platos.set_index("id_plato")["plato"].to_dict()
pop_vec = np.array([POP_PLATOS[plato_names[i]] for i in plato_ids], dtype=float)
pop_vec = pop_vec / pop_vec.sum()

precios = {name: PRECIOS_POR_PLATO[name]*IGV for name in PRECIOS_POR_PLATO}

ventas_rows = []
for _, crow in calendario_peru.iterrows():
    base_min, base_max = VENTAS_BASE_MIN_MAX
    base = np.random.randint(base_min, base_max+1)
    total = int(base * crow["factor_demanda"] * crow["factor_pandemia"])
    total = max(30, total)  # cota inferior
    # reparto multinomial por popularidad
    cantidades = np.random.multinomial(total, pop_vec)
    for pid, cant in zip(plato_ids, cantidades):
        if cant==0:
            continue
        nombre = plato_names[pid]
        pu = precios[nombre]
        ventas_rows.append([
            f"V{pid}{crow['fecha'].strftime('%Y%m%d')}",
            crow["fecha"].date(),
            pd.Timestamp(crow["fecha"]).strftime("%H:%M:%S"), # hora dummy
            pid,
            float(cant),
            float(pu),
            float(pu*cant),
            random.choice(["Efectivo","Tarjeta","Yape"]),
            random.choice(["Ana","Luis","María","José"]),
            random.choice(["Mesa","Delivery","Take-out"]),
            "".join(random.choices(string.digits,k=8)),
            ""
        ])

ventas = pd.DataFrame(ventas_rows, columns=[
    "id_venta","fecha","hora","id_plato","cantidad","precio_unitario","precio_total",
    "forma_de_pago","responsable_de_venta","canal","numero_comprobante","datos_cliente"
])

# -------------------------
# 7) Capacidad diaria de work centers (calendar)
# -------------------------
wc_cal_rows = []
for _, row in calendario_peru.iterrows():
    fecha = row["fecha"]
    for _, w in work_center.iterrows():
        # eficiencia base ±5% en feriados/eventos
        eff = w["eficiencia"]
        if row["feriado"]==1 or row["evento"]!="":
            eff = eff * np.random.uniform(0.95,1.05)
        min_total = w["min_disp_dia"] * w["unidades_paralelas"] * eff
        wc_cal_rows.append([
            fecha.date(), fecha.year, w["work_center_id"], w["work_center_name"], w["linea"],
            w["min_disp_dia"], w["unidades_paralelas"], round(eff,3), round(min_total,1)
        ])
work_center_calendar = pd.DataFrame(wc_cal_rows, columns=[
    "fecha","anio","work_center_id","work_center_name","linea","min_disp_dia_base","unidades_paralelas","eficiencia","min_disp_total"
])

# -------------------------
# 8) PRODUCCIÓN (consumo insumo + merma) y control de capacidad
# -------------------------
# Precios de insumo (costo base) para compras
# (Se inicializa con heurística: FOOD_COST_PCT * precio_venta y distribuye por receta)
costo_insumo_base = dict()
for iid in insumo.id_insumo:
    # heurística simple si no se infiere por receta
    costo_insumo_base[iid] = np.random.uniform(4.0, 20.0)  # S/ por unidad de insumo (kg, L o unid)

# Índices rápidos
receta_by_plato = receta.groupby("id_plato")
ruta_by_plato = hoja_ruta.groupby("item_id")

produccion_rows = []
inv_mov_rows = []
consumo_wc_min_por_dia = defaultdict(lambda: defaultdict(float))  # fecha -> wc_id -> minutos

# Estado de lotes por insumo (lista FIFO: cada item = dict{idLote, cantidad, id_ubicacion, costo_unitario, fecha_vto})
lotes = defaultdict(list)

def nueva_compra(iid, fecha, cant_kg, costo_unitario, vida_util_dias=14):
    # genera lote y movimiento de entrada
    id_lote = f"L{iid}{fecha.strftime('%Y%m%d')}{''.join(random.choices(string.ascii_uppercase, k=2))}"
    id_ubic = asignar_ubicacion_insumo(iid)
    vto = fecha + timedelta(days=vida_util_dias)
    lotes[iid].append({"idLote":id_lote,"cantidad":cant_kg,"id_ubicacion":id_ubic,"costo":costo_unitario,"vto":vto})
    # Movimiento entrada
    inv_mov_rows.append([ # inventario_movimientos
        f"MOV-{id_lote}", fecha.date(), "Entrada", iid,
        insumo.loc[insumo.id_insumo==iid,"unidad"].iloc[0],
        float(cant_kg), id_lote, 1, id_ubic, "Compra", "OC", "Sistema", ""
    ])
    # Registro en compras (inventario_entrada_lotes)
    compras_rows.append([
        id_lote, fecha.date(), fecha.date(), None, iid,
        insumo.loc[insumo.id_insumo==iid,"unidad"].iloc[0],
        float(cant_kg), float(costo_unitario),
        14, vto.date(), 1, id_ubic, float(cant_kg),
        0.0, "Activo", ""
    ])
    return id_lote

# Contenedores compras e inventario diario
compras_rows = []
inv_diario_rows = []

# Demanda diaria de platos
ventas_dia = ventas.groupby(["fecha","id_plato"], as_index=False).agg(cant=("cantidad","sum"))

# Inicial: stock “semilla” por insumo (para arrancar sin quedarnos sin stock día 1)
for iid in insumo.id_insumo:
    # 2 a 5 kg/L o 10-30 unid según unidad
    u = insumo.loc[insumo.id_insumo==iid,"unidad"].iloc[0]
    base = np.random.uniform(2,5) if u in ["kg","L"] else np.random.uniform(10,30)
    cu = np.random.uniform(6.0,18.0)
    nueva_compra(iid, pd.to_datetime(START_DATE), base, cu, vida_util_dias=14)

# Función consumo FIFO
def consumir_fifo(iid, cantidad, fecha, ref, motivo):
    restante = cantidad
    total_extraido = 0.0
    while restante>1e-9 and lotes[iid]:
        lt = lotes[iid][0]
        extrae = min(lt["cantidad"], restante)
        lt["cantidad"] -= extrae
        restante -= extrae
        total_extraido += extrae
        # Movimiento salida
        inv_mov_rows.append([
            f"MOV-{lt['idLote']}-{ref}", fecha.date(), motivo, iid,
            insumo.loc[insumo.id_insumo==iid,"unidad"].iloc[0],
            -float(extrae), lt["idLote"], 1, lt["id_ubicacion"], "Producción", ref, "Cocina", ""
        ])
        if lt["cantidad"]<=1e-9:
            lotes[iid].pop(0)
    return total_extraido

# Política de reabastecimiento simple (cobertura objetivo)
consumo_hist = defaultdict(float)  # acumulado por insumo
dias_contados = defaultdict(int)

# Simula día a día
for fecha in pd.date_range(START_DATE, END_DATE, freq="D"):
    # Demanda por plato en el día
    dsel = ventas_dia[ventas_dia["fecha"]==pd.to_datetime(fecha.date())]
    # 1) Calcular demanda de insumos y tiempo en work centers
    demanda_insumo = defaultdict(float)
    tiempo_wc_min = defaultdict(float)
    for _, rowp in dsel.iterrows():
        pid = int(rowp["id_plato"]); cant_platos = float(rowp["cant"])
        # Ruta → tiempo
        if pid in ruta_by_plato.groups:
            for _, rr in ruta_by_plato.get_group(pid).iterrows():
                tiempo_wc_min[rr["work_center_id"]] += rr["run_time_min_por_unidad"] * cant_platos
        # Receta → insumos
        if pid in receta_by_plato.groups:
            for _, ri in receta_by_plato.get_group(pid).iterrows():
                # convertir unidades si fuera necesario (aquí dejamos conforme a unidad del insumo)
                demanda_insumo[int(ri["id_insumo"])] += float(ri["cantidad"]) * cant_platos

    # 2) Chequeo de capacidad (RCCP simple): si excede min_disp_total, recortar proporcional
    wc_cap = work_center_calendar[work_center_calendar["fecha"]==fecha.date()]
    cap_dict = dict(zip(wc_cap["work_center_id"], wc_cap["min_disp_total"]))
    # factor de recorte por cuello de botella
    recorte = 1.0
    for wc_id, tmin in tiempo_wc_min.items():
        if wc_id in cap_dict and cap_dict[wc_id] > 0:
            recorte = min(recorte, cap_dict[wc_id]/tmin) if tmin>0 else recorte
    recorte = min(1.0, recorte)
    # aplicar recorte a demanda_insumo y tiempos si hay cuello
    if recorte < 1.0:
        for k in demanda_insumo: demanda_insumo[k] *= recorte
        for k in tiempo_wc_min:  tiempo_wc_min[k]  *= recorte

    # 3) Consumir FIFO + registrar merma de producción
    for iid, req in demanda_insumo.items():
        # merma de producción aleatoria
        merma = req * np.random.uniform(MERMA_PROD_MIN, MERMA_PROD_MAX)
        total_salida = req + merma
        extraido = consumir_fifo(iid, total_salida, fecha, f"PD-{fecha.strftime('%Y%m%d')}", "Salida")
        # registrar merma explícita
        if merma>0:
            inv_mov_rows.append([
                f"MOV-MER-{iid}-{fecha.strftime('%Y%m%d')}", fecha.date(), "Merma", iid,
                insumo.loc[insumo.id_insumo==iid,"unidad"].iloc[0],
                -float(merma), None, 1, None, "Desperdicio", "PROD", "Cocina", ""
            ])
        # Producción (registro por insumo agregado al día)
        # (Para traza detallada por pedido sería a nivel plato, aquí dejamos resumen diario)
        produccion_rows.append([
            f"PD-{fecha.strftime('%Y%m%d')}", None, fecha.date(),
            None, None, None, iid, float(req),
            insumo.loc[insumo.id_insumo==iid,"unidad"].iloc[0],
            None, None, "Sistema", None, "Terminado", float(merma), ""
        ])
        # acumular consumo para política de reabastecimiento
        consumo_hist[iid] += req
    for wc_id, tmin in tiempo_wc_min.items():
        consumo_wc_min_por_dia[fecha.date()][wc_id] += tmin

    # 4) Reposición/Compras por insumo si cobertura < objetivo
    for iid in insumo.id_insumo:
        dias_contados[iid]+=1
        cons_prom_dia = consumo_hist[iid]/max(1, dias_contados[iid])
        # stock disponible actual
        stock_actual = sum([l["cantidad"] for l in lotes[iid]]) if lotes[iid] else 0.0
        # cobertura en días
        cobertura = stock_actual/cons_prom_dia if cons_prom_dia>0 else COBERTURA_OBJ_DIAS
        if cobertura < COBERTURA_OBJ_DIAS*0.6:
            # comprar hasta llegada a cobertura objetivo
            objetivo = COBERTURA_OBJ_DIAS*cons_prom_dia
            a_comprar = max(0.0, objetivo - stock_actual)
            # tamaño de lote mínimo por clase (kg/L/unid)
            u = insumo.loc[insumo.id_insumo==iid,"unidad"].iloc[0]
            min_lote = 8.0 if u in ["kg","L"] else 24.0
            cant = math.ceil(a_comprar / min_lote) * min_lote
            if cant>0:
                cu = np.random.uniform(0.8,1.2)*costo_insumo_base[iid]
                nueva_compra(iid, pd.to_datetime(fecha), cant, cu, vida_util_dias=14)

    # 5) Inventario diario (snapshot por lote vigente) + mermas por vencimiento mensual aprox
    for iid in insumo.id_insumo:
        # merma por almacenamiento aprox una vez cada ~30 días
        if fecha.day==28:
            if lotes[iid]:
                total = sum([l["cantidad"] for l in lotes[iid]])
                merm = total * MERMA_ALM_MENSUAL
                if merm>0:
                    _ = consumir_fifo(iid, merm, fecha, f"MA-{fecha.strftime('%Y%m%d')}", "Merma")
        if lotes[iid]:
            for l in lotes[iid]:
                stock_util = max(0.0, l["cantidad"]) if fecha<=l["vto"] else 0.0
                stock_venc = max(0.0, l["cantidad"]) if fecha>l["vto"] else 0.0
                inv_diario_rows.append([
                    f"REG-{iid}-{l['idLote']}-{fecha.strftime('%Y%m%d')}",
                    fecha.date(), iid, l["idLote"],
                    insumo.loc[insumo.id_insumo==iid,"unidad"].iloc[0],
                    1, l["id_ubicacion"], float(stock_util), float(stock_venc),
                    float(stock_util+stock_venc),
                    float(consumo_hist[iid]/max(1,dias_contados[iid]))*STOCK_MIN_DIAS,
                    0, almacen.loc[almacen.id_almacen==1,"capacidad_total_kg"].iloc[0],
                    0.0
                ])

# -------------------------
# 9) COMPRAS → Proveedores (derivados de historial)
# -------------------------
compras = pd.DataFrame(compras_rows, columns=[
    "id_lote","fecha_llegada","fecha_registro","id_proveedor","id_insumo","unidad","cantidad_entrada","costo_unitario",
    "vida_util_dias","fecha_vencimiento","id_almacen","id_ubicacion","stock_actual","Capacidad_utilizada_%","Estado_lote","observaciones"
])

# Generar proveedores automáticamente a partir de compras (vista derivada)
nombres_fake = ["Mercado Central","Avícola Surquillo","Pesquera San Pedro","Mayorista La Parada","Hortalizas Rimac","Lácteos Andinos","Huevos Villa"]
# asigna proveedor por insumo para consistencia
prov_map = {}
for iid in insumo.id_insumo:
    prov_map[iid] = random.choice(nombres_fake)

compras["id_proveedor"] = compras["id_insumo"].map(lambda x: abs(hash(prov_map[x])) % 10000 + 1)
# Create a mapping from generated provider ID back to provider name
prov_id_to_name = {abs(hash(name)) % 10000 + 1: name for name in nombres_fake}

proveedor = (compras[["id_proveedor"]].drop_duplicates()
             .assign(nombre_proveedor=lambda d: d["id_proveedor"].map(lambda x: prov_id_to_name.get(x, "Proveedor")),
                     ruc=lambda d: d["id_proveedor"].map(lambda x: "".join(random.choices(string.digits,k=11))),
                     telefono=lambda d: d["id_proveedor"].map(lambda x: "9"+"".join(random.choices(string.digits,k=8))),
                     email=lambda d: d["id_proveedor"].map(lambda x: f"contacto{x}@proveedor.pe"),
                     tipo="Mayorista", distrito="Lima", numero_contacto=lambda d: d["telefono"]))

# Vista proveedor_x_insumo
proveedor_x_insumo = (compras.groupby(["id_proveedor","id_insumo"], as_index=False)
                      .agg(precio_promedio=("costo_unitario","mean"),
                           tiempo_entrega_dias=("vida_util_dias","mean"),
                           confiabilidad_pct=("costo_unitario", lambda s: np.clip(100 - s.std()*5, 60, 99)))
                     )

# -------------------------
# 10) PLANIFICACIÓN (compras del siguiente período)
# -------------------------
# Forecast (aquí simple: promedio móvil 14d) → por plato
ventas_plato_d = ventas.groupby(["fecha","id_plato"], as_index=False).agg(q=("cantidad","sum"))
ventas_plato_d = ventas_plato_d.sort_values(["id_plato","fecha"])
ventas_plato_d["q_ma14"] = ventas_plato_d.groupby("id_plato")["q"].transform(lambda s: s.rolling(14,min_periods=7).mean())
last_day = ventas_plato_d["fecha"].max()

plan_rows = []
pland_rows = []

for pid in platos.id_plato:
    sub = ventas_plato_d[ventas_plato_d["id_plato"]==pid]
    vh = sub["q"].tail(14).mean() if len(sub)>=1 else 0.0
    fcast = sub["q_ma14"].iloc[-1] if (len(sub) and not np.isnan(sub["q_ma14"].iloc[-1])) else vh
    platos_planif = max(0.0, fcast)  # día siguiente
    # insumos requeridos
    req_insumos = receta[receta["id_plato"]==pid][["id_insumo","cantidad"]].copy()
    req_insumos["req_dia"] = req_insumos["cantidad"] * platos_planif

    # cobertura e inventario disponible por insumo
    for _, r in req_insumos.iterrows():
        iid = int(r["id_insumo"]); req = float(r["req_dia"])
        stock_act = 0.0
        if lotes[iid]:
            stock_act = sum([lt["cantidad"] for lt in lotes[iid]])
        cons_prom = consumo_hist[iid]/max(1, dias_contados[iid])
        a_comprar = max(0.0, COBERTURA_OBJ_DIAS*cons_prom - stock_act)
        lotes_vencer_7 = 0
        if lotes[iid]:
            lotes_vencer_7 = sum([1 for lt in lotes[iid] if 0<= (lt["vto"] - pd.Timestamp(fecha)+timedelta(days=1)).days <=7])
        # bloqueo por capacidad si RCCP recortó en el último día
        fecha_last = last_day
        wc_last = consumo_wc_min_por_dia.get(fecha_last, {})
        bloqueado = 0
        if wc_last:
            for wc_id, tmin in wc_last.items():
                cap = work_center_calendar[(work_center_calendar["fecha"]==fecha_last) & (work_center_calendar["work_center_id"]==wc_id)]
                if len(cap) and tmin > cap["min_disp_total"].iloc[0]*1.01:
                    bloqueado = 1; break

        # fila detalle
        pland_rows.append([
            len(pland_rows)+1, None, iid, round(req,3),
            round(stock_act,3), round(a_comprar,3),
            round(stock_act/cons_prom,2) if cons_prom>0 else COBERTURA_OBJ_DIAS,
            bool(bloqueado), int(lotes_vencer_7),
            round(a_comprar * np.mean(compras.loc[compras["id_insumo"]==iid,"costo_unitario"])) if (iid in compras["id_insumo"].values) else 0.0
        ])

    # cabecera plan por plato
    plan_rows.append([
        len(plan_rows)+1, (last_day+timedelta(days=1)), pid,
        round(vh,2), round(fcast,2), round(platos_planif,2),
        "(calculado en detalle)", None, None, False, None, None
    ])

planificacion = pd.DataFrame(plan_rows, columns=[
    "id_plan","fecha","id_plato","Ventas_hist","Proyección_demanda","Platos_planificados",
    "Insumos_a_comprar_estimado","Cobertura_días","Lotes_por_vencer_7d","Bloqueo_por_capacidad",
    "Costos_proyectados","Costos_reales"
])

planificacion_detalle_insumo = pd.DataFrame(pland_rows, columns=[
    "id_plan_detalle","id_plan","id_insumo","cantidad_requerida",
    "inventario_disponible","a_comprar","cobertura_dias","bloqueo_capacidad",
    "lotes_por_vencer_7d","costo_estimado"
])
# vincular id_plan por la fecha (1er plan por simplicidad)
if len(planificacion):
    planificacion_detalle_insumo["id_plan"] = planificacion["id_plan"].iloc[0]

# -------------------------
# 11) PRODUCCIÓN (formato de tu tabla, ya insertamos arriba en agregados)
# Para mantener columnas exactas:
produccion = pd.DataFrame(produccion_rows, columns=[
    "pedido_ID","linea_ID","fecha","hora_inicio_preparacion","hora_fin_preparacion",
    "id_plato","id_insumo","cantidad_insumo","unidad","id_lote","id_ubicacion","responsable",
    "tiempo_total","estado_pedido","desperdicio_registrado","observaciones"
])

# -------------------------
# 12) INVENTARIO_MOVIMIENTOS & INVENTARIO_DIARIO dataframes finales
inventario_movimientos = pd.DataFrame(inv_mov_rows, columns=[
    "id_mov","fecha","tipo_mov","id_insumo","unidad","cantidad","id_lote","id_almacen","id_ubicacion",
    "destino_origen","Documento_ref","Responsable","Observaciones"
])
inventario_diario = pd.DataFrame(inv_diario_rows, columns=[
    "id_reg","fecha","id_insumo","id_lote","unidad","id_almacen","id_ubicacion",
    "stock_utilizable","stock_vencido","stock_total","stock_minimo","Alerta_stock_mínimo",
    "Capacidad_total_kg","Capacidad_usada_%"
])

# -------------------------
# 13) VENTAS final ya está en df 'ventas'
# -------------------------

# -------------------------
# 14) EXPORTAR TODO A CSV
# -------------------------
def tocsv(df, name):
    path = f"{OUT_DIR}/{name}.csv"
    df.to_csv(path, index=False)
    return path

paths = []
paths += [tocsv(lineas, "linea")]
paths += [tocsv(platos, "plato")]
paths += [tocsv(insumo, "insumo")]
paths += [tocsv(receta, "receta")]
paths += [tocsv(almacen, "almacen")]
paths += [tocsv(almacen_ubicacion, "almacen_ubicacion")]
paths += [tocsv(work_center, "work_center")]
paths += [tocsv(hoja_ruta, "hoja_ruta")]
paths += [tocsv(rccp_standard, "rccp_standard")]
paths += [tocsv(work_center_calendar, "work_center_calendar")]
paths += [tocsv(calendario_peru, "calendario_peru")]
paths += [tocsv(ventas, "ventas")]
paths += [tocsv(produccion, "produccion")]
paths += [tocsv(pd.DataFrame(compras_rows, columns=[
    "id_lote","fecha_llegada","fecha_registro","id_proveedor","id_insumo","unidad","cantidad_entrada","costo_unitario",
    "vida_util_dias","fecha_vencimiento","id_almacen","id_ubicacion","stock_actual","Capacidad_utilizada_%","Estado_lote","observaciones"
]), "compras")]
paths += [tocsv(inventario_movimientos, "inventario_movimientos")]
paths += [tocsv(inventario_diario, "inventario_diario")]
paths += [tocsv(proveedor, "proveedor")]
paths += [tocsv(proveedor_x_insumo, "proveedor_x_insumo")]
paths += [tocsv(planificacion, "planificacion")]
paths += [tocsv(planificacion_detalle_insumo, "planificacion_detalle_insumo")]

print("✅ CSVs generados en:", OUT_DIR)
for p in paths: print(" -", p)

✅ CSVs generados en: data/raw
 - data/raw/linea.csv
 - data/raw/plato.csv
 - data/raw/insumo.csv
 - data/raw/receta.csv
 - data/raw/almacen.csv
 - data/raw/almacen_ubicacion.csv
 - data/raw/work_center.csv
 - data/raw/hoja_ruta.csv
 - data/raw/rccp_standard.csv
 - data/raw/work_center_calendar.csv
 - data/raw/calendario_peru.csv
 - data/raw/ventas.csv
 - data/raw/produccion.csv
 - data/raw/compras.csv
 - data/raw/inventario_movimientos.csv
 - data/raw/inventario_diario.csv
 - data/raw/proveedor.csv
 - data/raw/proveedor_x_insumo.csv
 - data/raw/planificacion.csv
 - data/raw/planificacion_detalle_insumo.csv


In [3]:
import pandas as pd
import os

OUT_DIR = "data/raw"
# List of generated CSV files (assuming they are in the OUT_DIR)
csv_files = [f for f in os.listdir(OUT_DIR) if f.endswith('.csv')]

for file_name in csv_files:
    file_path = os.path.join(OUT_DIR, file_name)
    print(f"--- {file_name} ---")
    try:
        df = pd.read_csv(file_path)
        print("\nHead:")
        display(df.head())
        print("\nData Types:")
        display(df.dtypes)
        print("-" * (len(file_name) + 8)) # Separator
    except Exception as e:
        print(f"Could not read {file_name}: {e}")

--- linea.csv ---

Head:


,id_linea,nombre_linea
0,1,Caliente
1,2,Fría
2,3,General



Data Types:


,0
id_linea,int64
nombre_linea,object


-----------------
--- almacen.csv ---

Head:


...

------------------
--- produccion.csv ---

Head:


,pedido_ID,linea_ID,fecha,hora_inicio_preparacion,hora_fin_preparacion,id_plato,id_insumo,cantidad_insumo,unidad,id_lote,id_ubicacion,responsable,tiempo_total,estado_pedido,desperdicio_registrado,observaciones



Data Types:


,0
pedido_ID,object
linea_ID,object
fecha,object
hora_inicio_preparacion,object
hora_fin_preparacion,object
id_plato,object
id_insumo,object
cantidad_insumo,object
unidad,object
id_lote,object


----------------------


# **Verificación de datos**

In [4]:
import pandas as pd

DIR = "data/raw"

compras = pd.read_csv(f"{DIR}/compras.csv", parse_dates=["fecha_llegada","fecha_registro","fecha_vencimiento"])
inv_mov = pd.read_csv(f"{DIR}/inventario_movimientos.csv", parse_dates=["fecha"])
inv_day = pd.read_csv(f"{DIR}/inventario_diario.csv", parse_dates=["fecha"])

# 1) ¿Todos los lotes de compras tienen la entrada?
lotes_compra = set(compras["id_lote"])
lotes_entrada = set(inv_mov.query("tipo_mov == 'Entrada'")["id_lote"])
print("Lotes en compras sin movimiento de Entrada:",
      len(lotes_compra - lotes_entrada))

# 2) ¿Los lotes aparecen en inventario diario?
lotes_inv_diario = set(inv_day["id_lote"].dropna())
print("Lotes en compras que NO aparecen en inventario_diario:",
      len(lotes_compra - lotes_inv_diario))


Lotes en compras sin movimiento de Entrada: 0
Lotes en compras que NO aparecen en inventario_diario: 0


In [5]:
# Elige un lote al azar
sample_lote = compras.sample(1, random_state=0)["id_lote"].iloc[0]
print("LOTE MUESTRA:", sample_lote)

mov_lote = inv_mov[inv_mov["id_lote"] == sample_lote].sort_values("fecha")
day_lote = inv_day[inv_day["id_lote"] == sample_lote].sort_values("fecha")

display(mov_lote.head(10))
display(day_lote.head(10))

# Saldos coherentes
entrada = mov_lote.query("tipo_mov=='Entrada'")["cantidad"].sum()
salidas = -mov_lote.query("tipo_mov=='Salida'")["cantidad"].sum()
mermas = -mov_lote.query("tipo_mov=='Merma'")["cantidad"].sum()
print("Entrada:", entrada, "Salidas:", salidas, "Mermas:", mermas)
print("Saldo teórico (Entrada - Salidas - Mermas):", entrada - salidas - mermas)
print("Stock diario mínimo/máximo observado:", day_lote["stock_total"].min(), day_lote["stock_total"].max())


LOTE MUESTRA: L12120210101AC


,id_mov,fecha,tipo_mov,id_insumo,unidad,cantidad,id_lote,id_almacen,id_ubicacion,destino_origen,Documento_ref,Responsable,Observaciones
20,MOV-L12120210101AC,2021-01-01,Entrada,121,kg,3.909782,L12120210101AC,1,1,Compra,OC,Sistema,NaN
42,MOV-L12120210101AC-MA-20210128,2021-01-28,Merma,121,kg,-0.058647,L12120210101AC,1,1,Producción,MA-20210128,Cocina,NaN
64,MOV-L12120210101AC-MA-20210228,2021-02-28,Merma,121,kg,-0.057767,L12120210101AC,1,1,Producción,MA-20210228,Cocina,NaN
86,MOV-L12120210101AC-MA-20210328,2021-03-28,Merma,121,kg,-0.056901,L12120210101AC,1,1,Producción,MA-20210328,Cocina,NaN
108,MOV-L12120210101AC-MA-20210428,2021-04-28,Merma,121,kg,-0.056047,L12120210101AC,1,1,Producción,MA-20210428,Cocina,NaN
130,MOV-L12120210101AC-MA-20210528,2021-05-28,Merma,121,kg,-0.055206,L12120210101AC,1,1,Producción,MA-20210528,Cocina,NaN
152,MOV-L12120210101AC-MA-20210628,2021-06-28,Merma,121,kg,-0.054378,L12120210101AC,1,1,Producción,MA-20210628,Cocina,NaN
174,MOV-L12120210101AC-MA-20210728,2021-07-28,Merma,121,kg,-0.053563,L12120210101AC,1,1,Producción,MA-20210728,Cocina,NaN
196,MOV-L12120210101AC-MA-20210828,2021-08-28,Merma,121,kg,-0.052759,L12120210101AC,1,1,Producción,MA-20210828,Cocina,NaN
218,MOV-L12120210101AC-MA-20210928,2021-09-28,Merma,121,kg,-0.051968,L12120210101AC,1,1,Producción,MA-20210928,Cocina,NaN


,id_reg,fecha,id_insumo,id_lote,unidad,id_almacen,id_ubicacion,stock_utilizable,stock_vencido,stock_total,stock_minimo,Alerta_stock_mínimo,Capacidad_total_kg,Capacidad_usada_%
20,REG-121-L12120210101AC-20210101,2021-01-01,121,L12120210101AC,kg,1,1,3.909782,0.0,3.909782,0.0,0,1500.0,0.0
42,REG-121-L12120210101AC-20210102,2021-01-02,121,L12120210101AC,kg,1,1,3.909782,0.0,3.909782,0.0,0,1500.0,0.0
64,REG-121-L12120210101AC-20210103,2021-01-03,121,L12120210101AC,kg,1,1,3.909782,0.0,3.909782,0.0,0,1500.0,0.0
86,REG-121-L12120210101AC-20210104,2021-01-04,121,L12120210101AC,kg,1,1,3.909782,0.0,3.909782,0.0,0,1500.0,0.0
108,REG-121-L12120210101AC-20210105,2021-01-05,121,L12120210101AC,kg,1,1,3.909782,0.0,3.909782,0.0,0,1500.0,0.0
130,REG-121-L12120210101AC-20210106,2021-01-06,121,L12120210101AC,kg,1,1,3.909782,0.0,3.909782,0.0,0,1500.0,0.0
152,REG-121-L12120210101AC-20210107,2021-01-07,121,L12120210101AC,kg,1,1,3.909782,0.0,3.909782,0.0,0,1500.0,0.0
174,REG-121-L12120210101AC-20210108,2021-01-08,121,L12120210101AC,kg,1,1,3.909782,0.0,3.909782,0.0,0,1500.0,0.0
196,REG-121-L12120210101AC-20210109,2021-01-09,121,L12120210101AC,kg,1,1,3.909782,0.0,3.909782,0.0,0,1500.0,0.0
218,REG-121-L12120210101AC-20210110,2021-01-10,121,L12120210101AC,kg,1,1,3.909782,0.0,3.909782,0.0,0,1500.0,0.0


Entrada: 3.9097820828549015 Salidas: -0.0 Mermas: 2.3309847577353526
Saldo teórico (Entrada - Salidas - Mermas): 1.5787973251195488
Stock diario mínimo/máximo observado: 1.5787973251195455 3.9097820828549015


In [6]:
import pandas as pd
import unicodedata

# === CONFIGURACIÓN ===
DIR = "data/raw"
FILE = f"{DIR}/inventario_movimientos.csv"

# === LECTURA ROBUSTA ===
# 1️⃣ Intenta leer con y sin BOM (algunos CSV tienen encabezado con \ufeff)
try:
    inv_mov = pd.read_csv(FILE, encoding='utf-8-sig', parse_dates=['fecha'])
except Exception:
    inv_mov = pd.read_csv(FILE, encoding='latin1', parse_dates=['fecha'])

# 2️⃣ Limpieza de nombres de columnas
def limpiar_columna(col):
    # quita espacios, normaliza tildes y minúsculas
    col = unicodedata.normalize("NFKD", col)
    return col.strip().lower().replace(" ", "_")

inv_mov.columns = [limpiar_columna(c) for c in inv_mov.columns]

print("🔍 Columnas detectadas:", inv_mov.columns.tolist())

# 3️⃣ Detección automática de la columna de cantidad
cand_cant = [c for c in inv_mov.columns if "cant" in c]
if not cand_cant:
    raise ValueError(f"No se encontró columna de cantidad en {FILE}. Revisa encabezados.")
cant_col = cand_cant[0]
print(f"✅ Usando columna '{cant_col}' como 'cantidad'.")

# 4️⃣ Validación básica
cols_requeridas = {"fecha","id_insumo",cant_col}
faltan = cols_requeridas - set(inv_mov.columns)
if faltan:
    raise ValueError(f"Faltan columnas requeridas: {faltan}")

# 5️⃣ Agrupación segura
salidas = (inv_mov[inv_mov["tipo_mov"].str.lower()=="salida"]
           .groupby(["fecha","id_insumo"], as_index=False)[cant_col]
           .sum()
           .rename(columns={cant_col:"consumo_real"}))

# 6️⃣ Normaliza signo (las salidas son negativas en el dataset)
salidas["consumo_real"] = -salidas["consumo_real"]

print("✅ Agrupación exitosa. Vista previa:")
display(salidas.head(10))


🔍 Columnas detectadas: ['id_mov', 'fecha', 'tipo_mov', 'id_insumo', 'unidad', 'cantidad', 'id_lote', 'id_almacen', 'id_ubicacion', 'destino_origen', 'documento_ref', 'responsable', 'observaciones']
✅ Usando columna 'cantidad' como 'cantidad'.
✅ Agrupación exitosa. Vista previa:


,fecha,id_insumo,consumo_real


In [7]:
wc_cal = pd.read_csv(f"{DIR}/work_center_calendar.csv", parse_dates=["fecha"])
# reconstruir consumo wc por día desde movimientos? (más directo: revisa flag de bloqueo en planificacion)
plan_det = pd.read_csv(f"{DIR}/planificacion_detalle_insumo.csv")

print("Veces con bloqueo_capacidad:", plan_det["bloqueo_capacidad"].sum())


Veces con bloqueo_capacidad: 0


In [8]:
plan = pd.read_csv(f"{DIR}/planificacion.csv", parse_dates=["fecha"])
plan_det = pd.read_csv(f"{DIR}/planificacion_detalle_insumo.csv")

display(plan.tail(3))
display(plan_det.sort_values("a_comprar", ascending=False).head(10))


,id_plan,fecha,id_plato,Ventas_hist,Proyección_demanda,Platos_planificados,Insumos_a_comprar_estimado,Cobertura_días,Lotes_por_vencer_7d,Bloqueo_por_capacidad,Costos_proyectados,Costos_reales
9,10,2026-01-01,10,5.64,5.64,5.64,(calculado en detalle),NaN,NaN,False,NaN,NaN
10,11,2026-01-01,11,3.57,3.57,3.57,(calculado en detalle),NaN,NaN,False,NaN,NaN
11,12,2026-01-01,12,3.29,3.29,3.29,(calculado en detalle),NaN,NaN,False,NaN,NaN


,id_plan_detalle,id_plan,id_insumo,cantidad_requerida,inventario_disponible,a_comprar,cobertura_dias,bloqueo_capacidad,lotes_por_vencer_7d,costo_estimado
0,1,1,101,4.616,1.780,0.0,7,False,0,0
1,2,1,102,3.846,1.570,0.0,7,False,0,0
2,3,1,103,1.026,0.809,0.0,7,False,0,0
3,4,1,116,0.256,1.100,0.0,7,False,0,0
4,5,1,117,0.513,1.045,0.0,7,False,0,0
5,6,1,104,9.200,1.068,0.0,7,False,0,0
6,7,1,105,4.600,1.452,0.0,7,False,0,0
7,8,1,106,2.300,1.307,0.0,7,False,0,0
8,9,1,117,0.460,1.045,0.0,7,False,0,0
9,10,1,107,3.557,1.401,0.0,7,False,0,0


In [ ]:
print(inv_mov.columns.tolist())


['id_mov', 'fecha', 'tipo_mov', 'id_insumo', 'unidad', 'cantidad', 'id_lote', 'id_almacen', 'id_ubicacion', 'destino_origen', 'Documento_ref', 'Responsable', 'Observaciones']


# **Actualizamos repositorio**

In [9]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	data/raw/almacen.csv
	data/raw/almacen_ubicacion.csv
	data/raw/calendario_peru.csv
	data/raw/compras.csv
	data/raw/hoja_ruta.csv
	data/raw/insumo.csv
	data/raw/inventario_diario.csv
	data/raw/inventario_movimientos.csv
	data/raw/linea.csv
	data/raw/planificacion.csv
	data/raw/planificacion_detalle_insumo.csv
	data/raw/plato.csv
	data/raw/produccion.csv
	data/raw/proveedor.csv
	data/raw/proveedor_x_insumo.csv
	data/raw/rccp_standard.csv
	data/raw/receta.csv
	data/raw/ventas.csv
	data/raw/work_center.csv
	data/raw/work_center_calendar.csv

nothing added to commit but untracked files present (use "git add" to track)


In [10]:
!git add .

In [13]:
!git config --global user.email "joseph7104@gmail.com"
!git config --global user.name "joseph7104"

In [19]:

# Importar la librería para acceder a los secrets
from google.colab import userdata
import os

# --- Reemplaza estos valores con los tuyos ---
username = "joseph7104"
repository = "-1INF46-Plan_Compras_Produccion"  # <-- Cambia esto por el nombre de tu repositorio
# ---------------------------------------------

# Obtener el token guardado en los secrets de Colab
token = userdata.get('TOKEN')

# Construir la URL del repositorio con el token de autenticación
# El formato es: https://@github.com//.git
repo_url_with_token = f"https://{token}@github.com/{username}/{repository}.git"

# Empujar los cambios al repositorio remoto usando la URL con el token
# Primero, eliminamos el 'origin' viejo para evitar conflictos
!git remote remove origin
# Luego, añadimos el nuevo 'origin' con el token
!git remote add origin {repo_url_with_token}
# Finalmente, hacemos el push
!git push origin master

Enumerating objects: 27, done.
Counting objects: 100% (27/27), done.
Delta compression using up to 2 threads
Compressing objects: 100% (24/24), done.
Writing objects: 100% (24/24), 534.12 KiB | 2.45 MiB/s, done.
Total 24 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/joseph7104/-1INF46-Plan_Compras_Produccion.git
   b9c409f..2f1ba9f  master -> master
